In [1]:
import sys
import os
import ee
import geemap
import rasterio
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors


try:
    # Initialize the library.
    ee.Authenticate()
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
    



Successfully saved authorization token.
Google Earth Engine has initialized successfully!


In [10]:
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
#     cvi = ee.Image().expression('(NIR/Green)*(Red/Green)', {
#         'NIR': input.select('B8').multiply(0.0001),
#         'Red': input.select('B4').multiply(0.0001),
#         'Green': input.select('B3').multiply(0.0001)
#       }).rename('SAVI')
#     evi = image.expression(
#       '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
#       'NIR' : image.select('B8').divide(10000),
#       'RED' : image.select('B4').divide(10000),
#       'BLUE': image.select('B2').divide(10000)}).rename('EVI')
    return image.addBands([ndvi, gndvi, savi])


In [4]:
Map1 = geemap.Map(location=[52.191838, 6.741434], zoom=13)
Map1.add_basemap('SATELLITE')
Map1


Map(center=[52.191838, 6.741434], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [5]:
fc = ee.FeatureCollection(Map1.draw_last_feature)
region = fc.geometry()
cords = region.getInfo()['coordinates']
cords
#%%


[[[6.83865, 52.214627],
  [6.834274, 52.213759],
  [6.826034, 52.208342],
  [6.833802, 52.205844],
  [6.836291, 52.206554],
  [6.837922, 52.205739],
  [6.844102, 52.205555],
  [6.845003, 52.207027],
  [6.846996, 52.208514],
  [6.846309, 52.209526],
  [6.846502, 52.210565],
  [6.843886, 52.212392],
  [6.841182, 52.212918],
  [6.83865, 52.214627]]]

In [6]:
region = ee.Geometry.Polygon([[[6.83865, 52.214627],
  [6.834274, 52.213759],
  [6.826034, 52.208342],
  [6.833802, 52.205844],
  [6.836291, 52.206554],
  [6.837922, 52.205739],
  [6.844102, 52.205555],
  [6.845003, 52.207027],
  [6.846996, 52.208514],
  [6.846309, 52.209526],
  [6.846502, 52.210565],
  [6.843886, 52.212392],
  [6.841182, 52.212918],
  [6.83865, 52.214627]]]
)

In [25]:
myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2019-04-01', '2019-08-31') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', listOfImages)


Number of images in the collection:  ['20190408T104029_20190408T104058_T31UGT', '20190408T104029_20190408T104058_T31UGU', '20190408T104029_20190408T104058_T32ULC', '20190513T104031_20190513T104026_T31UGT', '20190513T104031_20190513T104026_T31UGU', '20190513T104031_20190513T104026_T32ULC', '20190627T104029_20190627T104030_T31UGT', '20190627T104029_20190627T104030_T32ULC', '20190727T104029_20190727T104330_T31UGU', '20190821T104031_20190821T104025_T32ULC', '20190826T104029_20190826T104232_T31UGT', '20190826T104029_20190826T104232_T31UGU', '20190826T104029_20190826T104232_T32ULC']


In [26]:
filmArgs = {
    "dimensions": 128,
    "region": region,
    "crs": 'EPSG:3857',
    "min": 0.2, 
    "max" : 1,
    "palette":  ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400'],
#     "palette":  ['#006400', '#00c800', '#ffff00', '#ff0000', '#640000'],
#     "palette": ['white', 'green'],
#     "bands": ['B4', 'B3', 'B2'],
    "bands": ['NDVI'],
    "framesPerSecond": 1
}

print(myCollection.map(add_metrics).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25).getVideoThumbURL(filmArgs));


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/6973ba8d272e7fa1a4c4f3cb9d994be0-1f5532b5a3c71486c4eebaf48b35d9a5:getPixels
